In [8]:
import json
import datetime
import requests
from logging import getLogger
from dotenv import load_dotenv
from os import getenv
from fastapi import FastAPI
from pytz import timezone

# 从.env.local读取环境变量
load_dotenv(dotenv_path="../.env.local")

logger = getLogger(__name__)

app = FastAPI()

s = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/"
}

username = getenv("PKU_USERNAME")
password = getenv("PKU_PASSWORD")

r = s.post(
    "https://iaaa.pku.edu.cn/iaaa/oauthlogin.do",
    data={
        "appid": "wproc",
        "userName": username,
        "password": password,
        "redirUrl": "https://wproc.pku.edu.cn/site/login/cas-login?redirect_url=https://wproc.pku.edu.cn/v2/reserve/",
    },
)
token = json.loads(r.text)["token"]

r = s.get(
    f"https://wproc.pku.edu.cn/site/login/cas-login?redirect_url=https://wproc.pku.edu.cn/v2/reserve/&_rand=0.6441813796046802&token={token}"
)

date = datetime.datetime.now().strftime("%Y-%m-%d")

r = s.get(
    f"https://wproc.pku.edu.cn/site/reservation/list-page?hall_id=1&time=%{date}&p=1&page_size=0"
)
resources = json.loads(r.text)["d"]["list"]

r = s.get(
    "https://wproc.pku.edu.cn/site/reservation/launch",
    data={
        "resource_id": "4",
        "data": '[{"date": "2024-07-06", "period": 31, "sub_resource_id": 0}]',
    },
)

r = s.get(
    "https://wproc.pku.edu.cn/site/reservation/my-list-time?p=1&page_size=10&status=2&sort_time=true&sort=asc",
)
apps = json.loads(r.text)["d"]["data"]

app_0 = apps[0]
app_id = app_0["id"]
app_appointment_id = app_0["hall_appointment_data_id"]
r = s.get(
    f"https://wproc.pku.edu.cn/site/reservation/get-sign-qrcode?type=1&id=${app_id}&hall_appointment_data_id=${app_appointment_id}"
)
print(r.status_code)
print(r.text)


200
{"e":0,"m":"操作成功","d":{"name":"夏一飞\r\n1700011620\r\n物理学院","code":"iKpIT6RrwpCx4tROgDAP%2BjZlN2UxMmFmYjM1ZmMwZDlkMmMzNDMzZjlkMTY4OTE4OGM1N2JjZjRlYTQwYTcwNDNjNzlkYWM2ZDY4Mjk2Zjmhoq%2B0abGd188QGfW9ctWDJVqzMYsradVat7Tq89PMSs3bXNhVcH2LwNr9neazW8E4WA%2BnlqO9pa9l3Ri4MW%2FjVU%2FfSt1XbQQuYPC0oaaaZg%3D%3D"}}


In [9]:
import qrcode

# 输入需要生成二维码的字符串
data = "iKpIT6RrwpCx4tROgDAP%2BjZlN2UxMmFmYjM1ZmMwZDlkMmMzNDMzZjlkMTY4OTE4OGM1N2JjZjRlYTQwYTcwNDNjNzlkYWM2ZDY4Mjk2Zjmhoq%2B0abGd188QGfW9ctWDJVqzMYsradVat7Tq89PMSs3bXNhVcH2LwNr9neazW8E4WA%2BnlqO9pa9l3Ri4MW%2FjVU%2FfSt1XbQQuYPC0oaaaZg%3D%3D"

# 创建二维码对象
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)

# 添加数据到二维码对象
qr.add_data(data)
qr.make(fit=True)

# 创建图像
img = qr.make_image(fill='black', back_color='white')

# 保存图像
img.save("qrcode.png")